<a href="https://colab.research.google.com/github/awolaja/ChatGPT_Trading_Bot/blob/main/Rising_Rates_with_Vol_Switch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install alpaca-trade-api

In [46]:
!pip install --upgrade alpaca-trade-api


In [55]:
import pandas as pd
import yfinance as yf
import numpy as np
import alpaca_trade_api as tradeapi
from datetime import datetime, timedelta
#from alpaca_trade_api import REST

In [56]:
# Define your Alpaca API key and secret key
api_key = "PKB5WOUADWU7FATFGCYO"
api_secret = "s5SHSPqQ7NdZdS5xIGNrb1PeOeSY6MIJ6csRy9Fp"
BASE_URL = 'https://paper-api.alpaca.markets'

In [57]:
# Initialize the Alpaca API connection
api = tradeapi.REST(api_key, api_secret, base_url=BASE_URL)

In [58]:
# getting historical data
def get_historical_data(symbol, timeframe='1Day', limit=100):
    barset = api.get_bars(symbol, timeframe, limit=limit).df
    return barset

In [59]:
# calcuting rsi
def calculate_rsi(data, period=14):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [60]:
# Define the reward function
def calculate_ma(data, period=25):
    return data['close'].rolling(window=period).mean()

In [61]:
def calculate_max_drawdown(close_prices):
    # This function will calculate the max drawdown in the last 10 days
    roll_max = close_prices.rolling(10, min_periods=1).max()
    daily_drawdown = close_prices/roll_max - 1.0
    max_drawdown = daily_drawdown.rolling(10, min_periods=1).min()
    return max_drawdown

In [62]:
def select_bottom_by_rsi(symbols, period=10):
    # Select the symbol with the lowest RSI
    min_rsi = float('inf')
    selected_symbol = None
    for symbol in symbols:
        data = get_historical_data(symbol, limit=period)
        rsi = calculate_rsi(data, period=period)[-1]
        if rsi < min_rsi:
            min_rsi = rsi
            selected_symbol = symbol
    return selected_symbol

In [63]:
def select_bottom_by_ma(symbols, period=25):
    # Select the symbol with the lowest moving average
    min_ma = float('inf')
    selected_symbol = None
    for symbol in symbols:
        data = get_historical_data(symbol, limit=period)
        ma = MA(data['close'], timeperiod=period)[-1]
        if ma < min_ma:
            min_ma = ma
            selected_symbol = symbol
    return selected_symbol

In [64]:
def place_order(symbol, qty, side, type='market', time_in_force='gtc'):
    print(f"Placing order: {side} {qty} shares of {symbol}")
    try:
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side=side,
            type=type,
            time_in_force=time_in_force
        )
    except Exception as e:
        print(f"An exception occurred: {e}")


In [69]:
def get_historical_data(symbol, timeframe='1Day', limit=100):
    # Calculate the start date
    start_date = (datetime.now() - timedelta(days=limit * 2)).isoformat()
    try:
        barset = api.get_bars(symbol, timeframe, start=start_date, limit=limit).df
        return barset
    except tradeapi.rest.APIError as e:
        print(f"Encountered an APIError: {e}")
        return None

In [70]:
# Function to execute orders
def place_order(symbol, qty, side):
    api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type='market',
        time_in_force='day'
    )

In [72]:
# Strategy execution function
def execute_strategy():
    try:
        qqq_data = get_historical_data('QQQ', limit=12)
        rsi_10 = calculate_rsi(qqq_data['close'], period=10).iloc[-1]
        rsi_12 = calculate_rsi(qqq_data['close'], period=12).iloc[-1]

        # Example: assign equal weight to each position, and assume 10,000 total capital
        account_info = api.get_account()
        total_capital = float(account_info.equity)
        position_size = total_capital / 5  # Example: 5 positions

        if rsi_10 > 79:
            # Buy UVXY
            qty = position_size // qqq_data['close'].iloc[-1]
            place_order('UVXY', qty, 'buy')
        elif rsi_10 > 32:
            # Buy TQQQ and BSV
            qty_tqqq = position_size // qqq_data['close'].iloc[-1]
            qty_bsv = position_size // get_historical_data('BSV', limit=1)['close'].iloc[-1]
            place_order('TQQQ', qty_tqqq, 'buy')
            place_order('BSV', qty_bsv, 'buy')
        elif rsi_12 > 6:
            # Buy DBMF and SQQQ
            qty_dbmf = position_size // get_historical_data('DBMF', limit=1)['close'].iloc[-1]
            qty_sqqq = position_size // get_historical_data('SQQQ', limit=1)['close'].iloc[-1]
            place_order('DBMF', qty_dbmf, 'buy')
            place_order('SQQQ', qty_sqqq, 'buy')
        else:
            tmf_data = get_historical_data('TMF', limit=10)
            max_drawdown_10d = calculate_max_drawdown(tmf_data['close']).iloc[-1]
            if max_drawdown_10d > -0.07:
                # Buy DBMF and TMV
                qty_dbmf = position_size // tmf_data['close'].iloc[-1]
                qty_tmv = position_size // get_historical_data('TMV', limit=1)['close'].iloc[-1]
                place_order('DBMF', qty_dbmf, 'buy')
                place_order('TMV', qty_tmv, 'buy')
            else:
                # Apply inverse volatility weighting to the specified ETFs
                # Assuming placeholder logic for a single ETF
                qty_tmf = position_size // tmf_data['close'].iloc[-1]
                place_order('TMF', qty_tmf, 'buy')
    except Exception as e:
        print(f"An error occurred: {e}")

# Execute the strategy
#

In [ ]:
if __name__ == "__main__":
    execute_strategy()